# Network Construction

In [1]:
import numpy as np
import pandas as pd
import pypsa
from pulp import *
import datetime

EU_Network = pypsa.Network()

EU_Network.import_from_netcdf('Network_Data/elec_s.nc')

Importing PyPSA from older version of PyPSA than current version 0.17.0.
Please read the release notes at https://pypsa.org/doc/release_notes.html
carefully to prepare your network for import.

INFO:pypsa.io:Imported network elec_s.nc has buses, carriers, generators, lines, links, loads, storage_units


## Adding Renewable Distribution

In [2]:
"""
Prepping Data
"""

Solar = pd.DataFrame(EU_Network.generators[EU_Network.generators['carrier'] == 'solar'])
Wind_Onshore = pd.DataFrame(EU_Network.generators[EU_Network.generators['carrier'] == 'onwind'])
Wind_Offshore_ac = pd.DataFrame(EU_Network.generators[EU_Network.generators['carrier'] == 'offwind-ac'])
Wind_Offshore_dc = pd.DataFrame(EU_Network.generators[EU_Network.generators['carrier'] == 'offwind-dc'])

All_Buses = pd.DataFrame(EU_Network.buses)
Countries = All_Buses['country'].unique()

#Data below taken from https://www.entsoe.eu/data/power-stats/net-gen-capacity/ 
#Where data was missing, data was taken from the following sources:
#Solar: IRENA renewable capacity statistics 2019 https://www.irena.org/-/media/Files/IRENA/Agency/Publication/2019/Mar/IRENA_RE_Capacity_Statistics_2019.pdf
#Wind: Wind Energy in Europe 2018 (Wind Europe) https://windeurope.org/wp-content/uploads/files/about-wind/statistics/WindEurope-Annual-Statistics-2018.pdf
#Verified EU renewable power plants from https://data.open-power-system-data.org/renewable_power_plants/ on 24/10/2020 (filtered for Wind and Solar)

Installed_Solar = np.load('Network_Data/Installed_Solar.npy',allow_pickle=True).item()
Installed_OnWind = np.load('Network_Data/Installed_OnWind.npy',allow_pickle=True).item()
Installed_OffWind = np.load('Network_Data/Installed_OffWind.npy',allow_pickle=True).item()

EU_RE = pd.read_csv('Network_Data/renewable_power_plants_EU.csv')
EU_RE.sort_values('electrical_capacity',ascending=False,inplace=True,ignore_index=True)
EU_RE.replace('UK','GB',inplace=True)

Solar_nodes = [i.split()[0] for i in Solar.index]
Wind_Onshore_nodes = [i.split()[0] for i in Wind_Onshore.index]
Wind_Offshore_ac_nodes = [i.split()[0] for i in Wind_Offshore_ac.index]
Wind_Offshore_dc_nodes = [i.split()[0] for i in Wind_Offshore_dc.index]
Wind_Offshore_nodes = list(set(Wind_Offshore_ac_nodes) | set(Wind_Offshore_dc_nodes)) 

In [3]:
def allocate(Nodes,longitude,latitude,capacity,country,gen_type):
    Buses = EU_Network.buses.loc[Nodes][EU_Network.buses.loc[Nodes]['country'] == country]
    Distances = np.sqrt(((Buses['x']-longitude)**2 + (Buses['y']-latitude)**2)).sort_values()
    if 'offwind' in gen_type:
        if Distances.iloc[0]*111 > 50: #assumes that generators more than 50 km from nearest node are dc linked, converts degree distance to km
            Nodes = Wind_Offshore_dc_nodes
            gen_type = 'offwind-dc'
        else:
            Nodes = Wind_Offshore_ac_nodes
            gen_type = 'offwind-ac'
        Buses = EU_Network.buses.loc[Nodes][EU_Network.buses.loc[Nodes]['country'] == country]
        Distances = np.sqrt(((Buses['x']-longitude)**2 + (Buses['y']-latitude)**2)).sort_values()
    Rank = 0
    while capacity > 0:
        generator = Distances.index[Rank]+' '+gen_type
        if EU_Network.generators.loc[generator,'p_nom_max'] - EU_Network.generators.loc[generator,'p_nom'] > capacity:
            EU_Network.generators.loc[generator,'p_nom'] += capacity
            capacity = 0
        else:
            start = EU_Network.generators.loc[generator,'p_nom']
            EU_Network.generators.loc[generator,'p_nom'] = EU_Network.generators.loc[generator,'p_nom_max']
            capacity -= EU_Network.generators.loc[generator,'p_nom'] - start
        Rank += 1

In [4]:
print(datetime.datetime.now())
for i in EU_RE.index:
    if EU_RE.loc[i,'energy_source_level_2'] == 'Solar':
        allocate(Solar_nodes,EU_RE.loc[i,'lon'],EU_RE.loc[i,'lat'],EU_RE.loc[i,'electrical_capacity'],EU_RE.loc[i,'country'],'solar')
    else:
        if EU_RE.loc[i,'technology'] == 'Offshore':
            allocate(Wind_Offshore_nodes,EU_RE.loc[i,'lon'],EU_RE.loc[i,'lat'],EU_RE.loc[i,'electrical_capacity'],EU_RE.loc[i,'country'],'offwind')
        else:
            allocate(Wind_Onshore_nodes,EU_RE.loc[i,'lon'],EU_RE.loc[i,'lat'],EU_RE.loc[i,'electrical_capacity'],EU_RE.loc[i,'country'],'onwind')
            
EU_Network.generators['p_nom_max'] -= EU_Network.generators['p_nom']
print(datetime.datetime.now())

2020-11-01 21:35:04.568891


INFO:numexpr.utils:NumExpr defaulting to 8 threads.


2020-11-02 03:02:50.332778


In [5]:
Solar = pd.DataFrame(EU_Network.generators[EU_Network.generators['carrier'] == 'solar'])
Wind_Onshore = pd.DataFrame(EU_Network.generators[EU_Network.generators['carrier'] == 'onwind'])
Wind_Offshore_ac = pd.DataFrame(EU_Network.generators[EU_Network.generators['carrier'] == 'offwind-ac'])
Wind_Offshore_dc = pd.DataFrame(EU_Network.generators[EU_Network.generators['carrier'] == 'offwind-dc'])
Tech_dfs = [Solar,Wind_Onshore,Wind_Offshore_ac,Wind_Offshore_dc]
Tech_name = ['solar','onwind','offwind-ac','offwind-dc']
Tech_nodes = [Solar_nodes,Wind_Onshore_nodes,Wind_Offshore_ac_nodes,Wind_Offshore_dc_nodes]
Tech_installed = [Installed_Solar,Installed_OnWind,Installed_OffWind]

for i in range(3):
    carrier = Tech_name[i]
    df = Tech_dfs[i]
    nodes = Tech_nodes[i]
    if i == 2:
        df = pd.concat([Tech_dfs[i],Tech_dfs[i+1]])
        carrier1 = Tech_name[i]
        carrier2 = Tech_name[i+1]
        nodes1 = Tech_nodes[i]
        nodes2 = Tech_nodes[i+1]
    for j in Countries:
        Buses = All_Buses[All_Buses['country'] == j].index
        country_nodes = list(set(Buses) & set(nodes))
        country_df = df.loc[[k+' '+carrier for k in country_nodes]]

        if i == 2:
            country_nodes1 = list(set(Buses) & set(nodes1))
            country_nodes2 = list(set(Buses) & set(nodes2))
            country_df = pd.concat([df.loc[[k+' '+carrier1 for k in country_nodes1]],df.loc[[k+' '+carrier2 for k in country_nodes2]]])
        
        IC = Tech_installed[i][j] - country_df['p_nom'].sum()
        if IC < 0:
            IC = 0    
            
        if len(country_nodes) > 0:
            LA = list(country_df['p_nom_max'])
            CF = [np.mean(np.array(EU_Network.generators_t.p_max_pu[k+' '+carrier])) for k in country_nodes]
            if i == 2:
                CF = np.concatenate(([np.mean(np.array(EU_Network.generators_t.p_max_pu[k+' '+carrier1])) for k in country_nodes1],
                                     [np.mean(np.array(EU_Network.generators_t.p_max_pu[k+' '+carrier2])) for k in country_nodes2]))

            # linear equation solving

            prob = LpProblem('LP', LpMinimize)
            alpha = LpVariable('alpha',lowBound=0)
            prob += 0, 'dummy objective function'
            prob += lpSum([alpha*LA[i]*CF[i]**3.5 for i in range(len(LA))]) == IC, 'summation constraint'
            
            for m in range(len(LA)):
                prob += alpha*CF[m]**3.5 <= 1, 'maximum LA constraint' + str(m)
            prob.solve(GUROBI(OutputFlag=0))    
            alpha = alpha.varValue
            
            if alpha is not None:
                for n in range(len(country_nodes)):
                    EU_Network.generators.p_nom_extendable[country_nodes[n]+' '+carrier] = False
                    EU_Network.generators.p_nom[country_nodes[n]+' '+carrier] += alpha*LA[n]*CF[n]**3.5
                if i == 2:
                    for n in range(len(country_nodes2)):
                        EU_Network.generators.p_nom_extendable[country_nodes2[n]+' '+carrier2] = False
                        EU_Network.generators.p_nom[country_nodes2[n]+' '+carrier2] += alpha*LA[n+len(country_nodes)]*CF[n+len(country_nodes)]**3.5
            else:
                print('Optimization failed for',carrier,'in',j)

Using license file C:\Users\brytni.johnston\gurobi.lic


INFO:gurobipy.gurobipy:Using license file C:\Users\brytni.johnston\gurobi.lic


Academic license - for non-commercial use only


INFO:gurobipy.gurobipy:Academic license - for non-commercial use only


Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2


C:\Users\brytni.johnston\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brytni.johnston\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status=

C:\Users\brytni.johnston\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brytni.johnston\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2
Gurobi status= 2


## Fix Bidding Zones
-Detach bidding zones in the Nordic countries and Italy

-Move Northern Ireland from UK to Ireland

-Join Luxembourg and Germany

In [6]:
NO = np.load('Network_Data/NO.npy',allow_pickle=True).item()
DK = np.load('Network_Data/DK.npy',allow_pickle=True).item()
SE = np.load('Network_Data/SE.npy',allow_pickle=True).item()
IT = np.load('Network_Data/IT.npy',allow_pickle=True).item()
IE = np.load('Network_Data/IE.npy',allow_pickle=True)

Zones = [NO,DK,SE,IT]

for i in Zones:
    for j in i.keys():
        for k in i[j]:
            EU_Network.buses['country'].loc[k] = j
for m in IE:       
    EU_Network.buses.loc[m,'country'] = 'IE'

for p in EU_Network.buses.index:
    if EU_Network.buses.loc[p,'country'] == 'DE':
        EU_Network.buses.loc[p,'country'] = 'DE/LU'
    if EU_Network.buses.loc[p,'country'] == 'LU':
        EU_Network.buses.loc[p,'country'] = 'DE/LU'  

C:\Users\brytni.johnston\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Add contingency generation

In [7]:
for node in EU_Network.buses.index:
    EU_Network.add('Generator','EmGenUp1{}'.format(node), bus=node,
                    p_nom=1000000, marginal_cost=1000000, carrier='Natural gas', p_max_pu=1, p_min_pu=0)
    EU_Network.add('Generator','EmGenUp2{}'.format(node), bus=node,
                    p_nom=1000000, marginal_cost=2000000, carrier='Natural gas', p_max_pu=0, p_min_pu=0)
    EU_Network.add('Generator','EmGenDown{}'.format(node), bus=node,
                    p_nom=1000000, marginal_cost=-2000000, carrier='Natural gas', p_max_pu=0, p_min_pu=0)
    
# Fix p_nom_max
EU_Network.links['p_nom_max'] = 1

## Export

In [8]:
Name = 'Output/EU_Network_created_' + str(datetime.date.today()) + '.nc'
EU_Network.export_to_netcdf(Name)

INFO:pypsa.io:Exported network EU_Network_created_2020-11-02.nc has generators, lines, buses, storage_units, carriers, links, loads


<xarray.Dataset>
Dimensions:                               (buses_i: 3037, carriers_i: 15, generators_i: 16013, generators_t_p_max_pu_i: 5827, lines_i: 4428, links_i: 26, loads_i: 2388, loads_t_p_set_i: 2388, snapshots: 8760, storage_units_i: 507, storage_units_t_inflow_i: 379)
Coordinates:
  * snapshots                             (snapshots) datetime64[ns] 2013-01-01 ... 2013-12-31T23:00:00
  * generators_i                          (generators_i) object '1086 offwind-ac' ... 'EmGenDown8067'
  * generators_t_p_max_pu_i               (generators_t_p_max_pu_i) object '1086 offwind-ac' ... '999 solar'
  * lines_i                               (lines_i) object '10236' ... '7308'
  * buses_i                               (buses_i) object '6964' ... '8067'
  * storage_units_i                       (storage_units_i) object '1261 PHS' ... '8017 PHS'
  * storage_units_t_inflow_i              (storage_units_t_inflow_i) object '1261 hydro' ... '7998 hydro'
  * carriers_i                            (carriers_i) object 'CCGT' ... 'ror'
  * links_i                               (links_i) object '4436' ... '11384+2'
  * loads_i                               (loads_i) object '1086' ... '999'
  * loads_t_p_set_i                       (loads_t_p_set_i) object '1086' ... '999'
Data variables:
    snapshots_weightings                  (snapshots) float64 1.0 1.0 ... 1.0
    generators_p_nom_max                  (generators_i) float64 3.495e+03 ... inf
    generators_weight                     (generators_i) float64 1.418e+03 ... nan
    generators_p_nom                      (generators_i) float64 0.0 ... 1e+06
    generators_capital_cost               (generators_i) float64 1.985e+05 ... 0.0
    generators_marginal_cost              (generators_i) float64 0.015 ... -2e+06
    generators_bus                        (generators_i) object '1086' ... '8067'
    generators_carrier                    (generators_i) object 'offwind-ac' ... 'Natural gas'
    generators_efficiency                 (generators_i) float64 1.0 1.0 ... 1.0
    generators_control                    (generators_i) object '' '' ... 'PQ'
    generators_p_nom_opt                  (generators_i) float64 nan nan ... 0.0
    generators_p_max_pu                   (generators_i) float64 1.0 1.0 ... 0.0
    generators_min_up_time                (generators_i) int64 0 0 0 0 ... 0 0 0
    generators_min_down_time              (generators_i) int64 0 0 0 0 ... 0 0 0
    generators_up_time_before             (generators_i) int64 1 1 1 1 ... 1 1 1
    generators_down_time_before           (generators_i) int64 0 0 0 0 ... 0 0 0
    generators_t_p_max_pu                 (snapshots, generators_t_p_max_pu_i) float64 0.03327 ... 0.0
    lines_bus0                            (lines_i) object '1622' ... '5597'
    lines_bus1                            (lines_i) object '6906' ... '8066'
    lines_v_nom                           (lines_i) float64 380.0 ... 380.0
    lines_num_parallel                    (lines_i) float64 0.3352 ... 2.0
    lines_length                          (lines_i) float64 11.51 ... 5.326
    lines_underground                     (lines_i) bool False False ... False
    lines_under_construction              (lines_i) bool False False ... False
    lines_tags                            (lines_i) object '' ... '"oid"=>"8482", "text_"=>" ", "symbol"=>"380-400 kV Transmission Line", "t9_code"=>"0", "visible"=>"1", "objectid"=>"2059.0", "tie_line"=>"0", "country_1"=>"DE", "country_2"=>"DE", "shape_leng"=>"41105.951525"'
    lines_geometry                        (lines_i) object 'LINESTRING(-0.850243688020123 37.898744605311,-0.836334000000007 37.937699,-0.758493882247685 37.9516994292935)' ... 'LINESTRING(12.3962611163051 53.2733664171279,12.31842 53.262749)'
    lines_type                            (lines_i) object 'Al/St 240/40 4-bundle 380.0' ... 'Al/St 240/40 4-bundle 380.0'
    lines_s_max_pu                        (lines_i) float64 0.7 0.7 ... 0.7 0.7
    lines_s_nom            